# VGChartz scrapper

In [1]:
import requests
import pandas as pd 
import numpy as np
import json
from bs4 import BeautifulSoup
import time
import seaborn as sns

In [2]:
def soup_request(response):
    '''soup and parse responses from a url request'''
    soup = BeautifulSoup(response.content)
    list_tags = soup.find('div',id = 'generalBody').findAll('tr')
    return list_tags

In [3]:
def parse_tag(tag_list):
    '''Given a list of table row's TAGS, Parse through soup to create dictionary '''
    final = {
        'Pos' : tag_list[0].text, #position in list,
        'Game': tag_list[2].a.text, #game
        'Console': tag_list[3].img['alt'] #console value hidden in img's alt value
    }
    #update the final dictionary with key column name, and parsed text value
    final.update({ key:value.text for key,value in zip(columns[3:],tag_list[4:])}) 
    return final

# Script

In [4]:
#287 pages
home = 'https://www.vgchartz.com/games/games.php?'
page = 1
params = "&results=200&name=&console=&keyword=&publisher=&genre=&order=Sales&ownership=Both&boxart=No&banner=No&showdeleted=&region=All&goty_year=&developer=&direction=DESC&showtotalsales=1&shownasales=1&showpalsales=1&showjapansales=1&showothersales=1&showpublisher=1&showdeveloper=0&showreleasedate=1&showlastupdate=1&showvgchartzscore=0&showcriticscore=1&showuserscore=0&showshipped=1&alphasort=&showmultiplat=No"
columns = ['Pos','Game','Console','Publisher','Critic Score','Total Shipped','Total Sales','NA Sales','PAL Sales','Japan Sales','Other Sales','Release Date','Last Update']

data = [] #result list of dictionarys

while page < 288: #288 is current max
    full_url = home+'page='+str(page)+params
    response = requests.get(full_url,time.sleep(2)) #request with time parameter
    
    if response:
        list_tags = soup_request(response)
        endlist = len(list_tags) - 1
        
        for tag in list_tags[3:endlist]:
            tags = tag.findAll('td')   #find all 'td' tags in tag
            data.append(parse_tag(tags))
            
    print('Finished page:'+str(page),sep = ' ') #short statement to see state of loop
    page += 1


Success on 1
Finished page:1
Success on 2
Finished page:2
Success on 3
Finished page:3
Success on 4
Finished page:4
Success on 5
Finished page:5
Success on 6
Finished page:6


KeyboardInterrupt: 

In [8]:
df = pd.DataFrame(data,columns = columns) #convert dta into dataframe
df.head()

,Pos,Game,Console,Publisher,Critic Score,Total Shipped,Total Sales,NA Sales,PAL Sales,Japan Sales,Other Sales,Release Date,Last Update
0,1,Dream C Club,PS3,D3Publisher,N/A,N/A,0.02m,N/A,N/A,0.02m,N/A,15th Nov 12,N/A
1,2,Vitamin X: Detective B6,PSP,D3Publisher,N/A,N/A,0.02m,N/A,N/A,0.02m,N/A,09th Feb 12,N/A
2,3,Nobunaga no Yabou Online: Houou no Shou,PS3,Tecmo Koei,N/A,N/A,0.01m,N/A,N/A,0.01m,N/A,07th Mar 12,N/A
3,4,Ninja Gaiden Sigma 2,PSV,Tecmo Koei,N/A,N/A,0.01m,N/A,0.00m,0.01m,0.00m,26th Feb 13,N/A
4,5,Shinobi Koutsutsu,PSP,Idea Factory,N/A,N/A,0.01m,N/A,N/A,0.01m,N/A,30th Jan 14,N/A


# Save

In [517]:
# df.to_csv('VGChartz.csv') #save dataframe to csv